<a href="https://colab.research.google.com/github/kckohkoh2/fantastic-disco/blob/master/day0412.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update

In [ ]:
!pip install mujoco

In [ ]:
!pip install pyvirtualdisplay
!sudo apt-get install xvfb
!pip install xvfbwrapper

In [ ]:
import gym
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
from gym.wrappers.record_video import RecordVideo
from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else:
    print("Could not find video")


env_name = 'Ant-v4'
env = gym.make(env_name, render_mode="rgb_array")
env = RecordVideo(env, './video',  episode_trigger = lambda episode_number: True)
state = env.reset()

while True:
  env.render()

  mu,sigma = agent.get_action(torch.from_numpy(state).float().to(device))
  dist = torch.distributions.Normal(mu,sigma[0])
  action = dist.sample()
  state, reward, done, info = env.step(action.cpu().numpy())

  if done: break

env.close()
show_video()

In [ ]:
pip install numpy==1.23.5


In [ ]:
import gym
print(gym.__version__)


In [ ]:
import gym
import glob
import io
import base64
import torch
import numpy as np
from IPython.display import HTML
from IPython import display as ipythondisplay
from gym.wrappers.record_video import RecordVideo
from pyvirtualdisplay import Display

# 가상 디스플레이 설정 (Colab 환경에서 필요)
display = Display(visible=0, size=(1400, 900))
display.start()

# 영상 표시 함수
def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

# 더미 에이전트 정의
class DummyAgent:
    def get_action(self, state):
        mu = torch.zeros(8).to(state.device)       # Ant-v4의 액션 공간은 8차원
        sigma = torch.ones(8).to(state.device) * 0.5
        return mu, sigma

# 에이전트 인스턴스 생성
agent = DummyAgent()

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 환경 생성 및 설정
env_name = 'Ant-v4'
env = gym.make(env_name, render_mode="rgb_array")
env = RecordVideo(env, './video', episode_trigger=lambda episode_number: True)

# 환경 초기화
state = env.reset()

# 에피소드 실행
while True:
    env.render()

    # 상태를 텐서로 변환하여 에이전트에 전달
    state_tensor = torch.from_numpy(state).float().to(device)
    mu, sigma = agent.get_action(state_tensor)
    dist = torch.distributions.Normal(mu, sigma)
    action = dist.sample().cpu().numpy()

    # 환경에 액션 적용
    state, reward, done, info = env.step(action)

    if done:
        break

# 환경 종료 및 영상 표시
env.close()
show_video()